In [1]:
from sklearn.decomposition import randomized_svd
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [2]:
# user = pd.read_csv('../Data/whyout_data/user.csv') # shape(31178,13), user idx에서 2354개가 비어있음
place = pd.read_csv('../Data/whyout_data/place.csv') # shape(4697,10), place idx에서 23개가 비어있음
# product = pd.read_csv('../Data/whyout_data/product.csv') # shape(5834,11), product idx에서 538개가 비어있음
# video = pd.read_csv('../Data/whyout_data/video.csv') # shape(3250, 9), video idx에서 315개가 비어있음
# user_place = pd.read_csv('../Data/whyout_data/user_place.csv') # shape (31176,4697) 유저 x 장소 아이템 # 8756명이 장소를 평가하지 않음
# user_product = pd.read_csv('../Data/whyout_data/user_product.csv') # shape(31176,5834) 유저 x 상품 아이템
# user_video = pd.read_csv('../Data/whyout_data/user_video.csv') # shape (31176, 3250) 유저 x 영상 아이템
#null_del_user_place = pd.read_csv('../Data/whyout_data/null_del_user_place.csv')
# null_del_user_product = pd.read_csv('../Data/whyout_data/null_del_user_product.csv')
# null_del_user_video = pd.read_csv('../Data/whyout_data/null_del_user_video.csv')
index_null_del_user_place = pd.read_csv('../Data/whyout_data/index_null_del_user_place.csv', index_col=0)
# index_null_del_user_product = pd.read_csv('../Data/whyout_data/index_null_del_user_product.csv', index_col=0)
# index_null_del_user_video = pd.read_csv('../Data/whyout_data/index_null_del_user_video.csv', index_col=0)
# normal_index_null_del_user_place = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_place.csv', index_col=0)
# normal_index_null_del_user_product = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_product.csv', index_col=0)
# normal_index_null_del_user_video = pd.read_csv('../Data/whyout_data/normal_index_null_del_user_video.csv', index_col=0)

In [5]:
def SVD(df, n_components=20):
    """
    사용자-상품 평점 행렬에 대해 특이값 분해(SVD)를 수행하고 예측 평점을 반환합니다.
    
    매개변수:
    user_product_matrix (numpy.ndarray): 사용자-상품 평점 행렬.
    n_components (int): 계산할 특이값 및 벡터의 수. 기본값은 20입니다.
    
    반환값:
    pd.DataFrame: 각 사용자와 상품에 대한 예측 평점을 포함하는 DataFrame.
    """
    # 데이터프레임을 numpy matrix로 만듦
    user_product_matrix = df.values

    # 각 사용자의 평균 평점을 계산함
    user_ratings_mean = np.mean(user_product_matrix, axis=1)
    
    # # 사용자 평균 평점을 뺀 행렬을 만듦
    # matrix_user_mean = user_product_matrix - user_ratings_mean.reshape(-1, 1)
    
    # U 행렬, sigma 행렬, V 전치 행렬을 반환.
    U, sigma, Vt = randomized_svd(user_product_matrix, n_components=n_components)
    
    # 특이값 벡터를 대각 행렬 형태로 변환해줌
    sigma = np.diag(sigma)
    
    # U, Sigma, Vt의 행렬곱을 수행하면, 다시 원본 행렬로 복원. 
    # 거기에 + 사용자 평균 rating을 적용
    svd_user_predicted_ratings = np.matmul(np.matmul(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
    
    # 예측된 행렬을 DataFrame으로 변환
    df_svd_preds = pd.DataFrame(svd_user_predicted_ratings, columns=df.columns)
    
    return df_svd_preds

predicted_ratings = SVD(index_null_del_user_place)
predicted_ratings.head()

,0,1,2,3,4,5,6,7,8,9,...,4687,4688,4689,4690,4691,4692,4693,4694,4695,4696
0,-5.910796e-29,-7.080079e-14,-1.855378e-14,1.784229e-14,-3.942599e-14,1.842008e-15,2.248079e-15,-4.903298e-15,-2.164392e-15,-2.385024e-30,...,0.000000,9.440684e-18,0.000000,-3.116406e-17,-7.450343e-17,6.514836e-18,1.715996e-19,1.715996e-19,2.592938e-17,0.000000
1,2.129019e-04,1.936967e-03,-6.301335e-04,-1.836498e-03,1.524329e-03,2.860985e-04,-4.072065e-05,3.260940e-02,3.483509e-04,2.129019e-04,...,0.000213,1.786858e-04,0.000213,2.132083e-04,2.127456e-04,1.779721e-04,2.128985e-04,2.128985e-04,2.054173e-04,0.000213
2,6.174154e-03,1.732482e-02,6.118355e-02,5.149496e-02,9.524995e-03,9.483547e-02,3.156961e-02,6.649649e-03,1.224148e-01,6.174154e-03,...,0.006174,1.012165e-02,0.006174,6.155362e-03,6.161038e-03,1.014772e-02,6.174050e-03,6.174050e-03,6.185284e-03,0.006174
3,2.876385e-29,2.643973e-14,-6.899032e-17,-3.989076e-15,2.133736e-15,-2.251659e-15,-3.145652e-16,5.030565e-15,1.331494e-15,-1.384908e-29,...,0.000000,-3.705714e-17,0.000000,1.272520e-17,2.455512e-17,-2.400489e-17,-4.960665e-20,-4.960665e-20,-3.532766e-18,0.000000
4,1.532893e-02,6.024906e-02,1.766172e-01,1.327316e-01,1.407461e-01,2.389637e-01,1.252697e-01,5.949185e-02,3.067453e-01,1.532893e-02,...,0.015329,2.423588e-02,0.015329,1.529015e-02,1.533591e-02,2.430564e-02,1.532849e-02,1.532849e-02,1.549769e-02,0.015329


In [6]:
def recommend_items(df_svd_preds, user_id, item_df, ratings_df, num_recommendations):
    """
    :param df_svd_preds: SVD 예측 평점 데이터프레임
    :param user_id: 사용자 ID
    :param df: 아이템 데이터프레임
    :param ratings_df: 사용자 평점 데이터프레임
    :param num_recommendations: 추천할 아이템 수
    :return: 사용자가 이미 평가한 아이템 인덱스와 추천 아이템 데이터프레임
    """
    # 원본 평점 데이터에서 user_id에 해당하는 행을 Series 형태로 가져오기
    user_data = ratings_df.loc[user_id]

    # 사용자가 이미 평가한 상품의 인덱스를 추출
    user_history_indices = [int(i) for i in user_data[user_data > 0].index.tolist()]

    # SVD를 통해 예측된 사용자의 상품 평점을 기반으로 상품 데이터를 정렬
    user_row_number = user_id - 1
    sorted_user_predictions = df_svd_preds.iloc[user_row_number].sort_values(ascending=False)

    # 사용자가 아직 평가하지 않은 상품 목록을 추출
    recommendations = item_df.loc[~item_df.index.isin(user_history_indices)]

    # 예측된 상품 평점 데이터와 원본 상품 데이터를 합침
    predictions_df = pd.DataFrame(sorted_user_predictions).reset_index()
    predictions_df['index'] = predictions_df['index'].astype(int)
    predictions_df.set_index('index', inplace=True)

    # recommendations의 인덱스와 predictions_df의 인덱스를 기준으로 병합
    recommendations = recommendations.merge(predictions_df, left_index=True, right_index=True)

    # 컬럼 이름을 바꾸고 정렬해서 반환
    recommendations = recommendations.rename(columns={user_row_number: 'Predictions'}).sort_values('Predictions', ascending=False).iloc[:num_recommendations]
    print(f"user {user_id}에게 추천해줄 {num_recommendations}개 아이템 id : {recommendations.index.values}")

    return user_history_indices, recommendations

# 함수 사용 예시
already_rated, predictions = recommend_items(predicted_ratings, 10, place, index_null_del_user_place, 10)

user 10에게 추천해줄 10개 아이템 id : [1384 3348 3457 3671 3283 2891 3427 1752 2265 3892]


In [7]:
predictions

,idx,지역(위치정보),공간유형,주변환경,계절,테마,클릭 수,좋아요 수,북마크 수,댓글 수,Predictions
1384,1385,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,1,0,1,0,1,1,0,1,0]",0.284178,0.324324,0.320352,0.000000,0.037386
3348,3350,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,0,1,0,0,0,0]",NaN,"[1,1,1,1,1,0,0,0,0,0,0,0,0,1,0]",0.326868,0.270270,0.326633,0.222222,0.036403
3457,3460,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,1,0,0,0,0,0,0,1,0]",0.254262,0.193694,0.236181,0.111111,0.030193
3671,3674,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[0,1,0,0,0,0,0,0]",NaN,"[1,1,1,1,0,0,1,0,1,0,0,1,0,0,1]",0.258442,0.225225,0.316583,0.000000,0.028433
3283,3284,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,0,1,0,0,1,0,0,0,0,1,0,1,0]",0.171512,0.090090,0.173367,0.000000,0.027173
2891,2892,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,1,0,0,0,0,0,0]",NaN,"[1,0,1,1,0,0,1,0,0,1,0,0,0,1,0]",0.215423,0.162162,0.164573,0.111111,0.024617
3427,3429,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,0,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,0,0,0,0,0,1,0,1,0]",0.197060,0.202703,0.228643,0.000000,0.023181
1752,1753,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,1,0,0,0,0,0,0]","[0,1,0,0,0,0,0,0]","[1,1,1,0]","[1,1,1,1,1,1,0,0,0,1,0,0,0,0,0]",0.213685,0.148649,0.157035,0.000000,0.022203
2265,2266,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,1,0,0,0,0,0,0]","[1,0,0,0,0,0,0,0]",NaN,"[1,1,1,1,1,0,0,0,1,1,0,1,1,1,0]",0.252759,0.202703,0.251256,0.000000,0.017877
3892,3900,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1,0,1,0,0,0,0,0,0]","[0,1,0,1,0,0,0,0]",NaN,"[1,1,1,1,1,0,1,0,0,0,1,0,0,0,1]",0.234068,0.153153,0.208543,0.111111,0.016463
